In [ ]:
# Importaciones
import pandas as pd
import matplotlib.pyplot as plt
pd.options.plotting.backend = "plotly"
import numpy as np
from sklearn import preprocessing
import seaborn as sns
from datetime import datetime
from IPython.core.display import display, HTML
from scipy import stats
display(HTML("<style>.container { width:100% !important; }</style>"))

# DEFINICIÓN DE FUNCIONES

Defino **add_lag**: función para hacer los corrimientos temporales en las variables.

In [ ]:
def add_lag(x,time_pre = 0, time_post = 0, delta_time = 0, replace_method = 'mean', delta_calc = 'absolute'):
    """
    This function calculate the difference between time_pre and time_post and shift the variable x
    If the delta_time is different to 0 then add_lag shift x in delta_time samples
    
    Time format: 'dd/mm/aaaa-hh:mm:ss'
    """
    from datetime import datetime
    # Defino valores para reemplazar las muestras vacías que se van a generar por el corrimiento
    if replace_method == 'mean':
        replaceValue = np.mean(x)
        
    if replace_method == 'zeros':
        replaceValue = 0.0
    
    # Realizo el corrimiento
    if delta_calc == 'absolute':
        delta = delta_time

        
    if delta_calc == 'dates':
        pre = datetime.strptime(time_pre, '%d/%m/%Y-%H:%M:%S')
        post = datetime.strptime(time_post, '%d/%m/%Y-%H:%M:%S')
        delta = (post-pre).seconds

    # Reemplazo los valores vacíos generados
    aux = x.shift(periods=delta)
    aux[:delta] =  replaceValue
    return aux

Defino **maximize_correlation**: función para maximizar la correlación entre dos funciones realizando corrimientos temporales.

In [ ]:
def maximize_correlation(fixedSignal, mobileSignal, maxLag):
    tempLagSignal = []
    corrVector = []
    corrVector2 = []
    
    # Guardo la correlación inicial
    initCorr = np.corrcoef(mobileSignal, fixedSignal)[0,1]
    
    # Encuentro la mayor correlación realizando saltos de 1 min
    for i in np.arange(1, maxLag):
        tempLagSignal = add_lag(mobileSignal, delta_time = (60 * i))
#        print('correlación:' + str(np.corrcoef(tempLagSignal, fixedSignal)[0,1]))
        
        corrVector.append(np.corrcoef(tempLagSignal, fixedSignal)[0,1])
        
    # Encuentro la mayor correlación en un lapso de 60 seg antes y después del valor de retardo obtenido anteriormente
    corrVector = np.array(corrVector)
    absCorrVector = abs(corrVector)  # Aplico Abs para encontrar correlaciones positivas y negativas 
    maxIndex = int(np.where(absCorrVector == absCorrVector.max())[0])   # Obtengo el índice donde se encuentra la correlación más alta
    
    if maxIndex != 0: # En caso de que el máximo esté en índice 0, buscará 120 segundos a partir del 0
        maxIndex = maxIndex - 1 # Le resto 1 para que empiece a buscar un minuto antes de la máxima correlación
    
    lagInit = maxIndex * 60    # Paso a segundos (samples) la posición de la máxima correlación

    for i in np.arange(0, 120):
        tempLagSignal = add_lag(mobileSignal, delta_time = (lagInit + i))
        corrVector2.append(np.corrcoef(tempLagSignal, fixedSignal)[0,1])
      
    
    
    absCorrVector2 = abs(np.array(corrVector2))  # Aplico Abs para encontrar correlaciones positivas y negativas
#     print(absCorrVector2)
#     print(np.where(absCorrVector2 == absCorrVector2.max()))
    maxIndex2 = int(np.where(absCorrVector2 == absCorrVector2.max())[0])   # Obtengo el índice donde se encuentra la correlación más alta
    lagMaxCorr = lagInit + maxIndex2
    
    finalLagSignal = add_lag(mobileSignal, delta_time = lagMaxCorr)
    maxCorrelation = np.corrcoef(finalLagSignal, fixedSignal)[0,1]
    
    print('El retardo que maximiza la correlación es de: ' + str(lagMaxCorr) + ' segundos (' + str(np.round(lagMaxCorr/60)) + ' minutos)')
    print('La correlación inicial era: ' + str(initCorr) + ' y luego del retardo es: ' + str(maxCorrelation))
    return finalLagSignal

Defino **plot_corrMatrix**: función para graficar en formato mapa de calor la matriz de correlaciones entre las variables del dataset.

In [ ]:
def plot_corrMatrix(data, method):
    import seaborn as sns
    corr = data.corr(method = method)
    mask = np.zeros_like(corr)
    mask[np.triu_indices_from(mask)] = True
    cmap = sns.diverging_palette(220, 10, as_cmap=True)
    fig, ax = plt.subplots(figsize=(20,17))
    sns.heatmap(corr, mask=mask, cmap=cmap, square=True,center=0, annot=True, ax=ax)

Defino **make_mi_scores**: función para graficar los aportes de cada variable utilizando el criterio de la información mutua.

In [ ]:
def make_mi_scores(X, y, discrete_features = 'auto'):
    from sklearn.feature_selection import mutual_info_regression
    mi_scores = mutual_info_regression(X, y, discrete_features=discrete_features)
    mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    
    mi_scores = mi_scores.sort_values(ascending=True)
    width = np.arange(len(mi_scores))
    ticks = list(mi_scores.index)
    plt.figure(dpi=100, figsize=(12, 7))
    plt.barh(width, mi_scores)
    plt.yticks(width, ticks)
    plt.title("Mutual Information Scores")

Defino **make_corr_scores**: función para graficar las correlaciones entre las variables de entrada X y la variable de salida y, utilizando Spearman, Kendall y Pearson.

In [ ]:
def make_corr_scores2(X, y):
    import scipy.stats
    corr_scores = []
    corr_scores = np.array(corr_scores)
    for i in X.columns:
        # print(corr_scores,np.corrcoef(X[i], y)[0,1])
        corr_scores = np.append(corr_scores,abs(np.corrcoef(X[i], y)[0,1]))
    corr_scores = pd.Series(corr_scores, name="MI Scores", index=X.columns)
    corr_scores = corr_scores.sort_values(ascending = True)
    width = np.arange(len(corr_scores))
    ticks = list(corr_scores.index)
    
    corr_scores2 = []
    corr_scores2 = np.array(corr_scores2)
    for i in X.columns:
        # print(corr_scores,np.corrcoef(X[i], y)[0,1])
        corr_scores2 = np.append(corr_scores2,abs(scipy.stats.spearmanr(X[i], y)[0]))
    corr_scores2 = pd.Series(corr_scores2, name="MI Scores", index=X.columns)
    corr_scores2 = corr_scores2.sort_values(ascending = True)
    width = np.arange(len(corr_scores2))
    ticks = list(corr_scores2.index)
    
    corr_scores3 = []
    corr_scores3 = np.array(corr_scores3)
    for i in X.columns:
        # print(corr_scores,np.corrcoef(X[i], y)[0,1])
        corr_scores3 = np.append(corr_scores3,abs(scipy.stats.kendalltau(X[i], y)[0]))
    corr_scores3 = pd.Series(corr_scores3, name = "MI Scores", index = X.columns)
    corr_scores3 = corr_scores3.sort_values(ascending = True)
    width = np.arange(len(corr_scores3))
    ticks = list(corr_scores3.index)
    
    
    plt.figure(dpi=100, figsize=(17, 10))
    plt.barh(width, corr_scores, label = 'Pearson R')
    plt.barh(width, corr_scores2,  label = 'Spearman Rho')
    plt.barh(width, corr_scores3, label = 'Kendall Tau')
    plt.yticks(width, ticks)
    plt.title("CORRELATION SCORES")
    plt.legend()
    
#     print(corr_scores)
#     return pd.DataFrame([X.columns, corr_scores.T])

Defino **MVA**: función que realiza el cálculo de la media móvil tomando una ventana de r puntos.

In [ ]:
def MVA(data, r):
    aux = []
    for i in np.arange(0, len(data)):
        aux.append(np.sum(data[i-r:i+r+1]) / (2*r + 1))
    return np.array(aux)

# IMPORTACIÓN DE LOS DATOS

In [ ]:
data=pd.read_csv(r'C:\Users\alexb\Documents\Facultad\UBA\Modelo_PGSM_3T\Datos\20210811 - Segunda corrida de datos\To_FE_20210811_2320_3T.csv', delimiter=',', sep='\n', decimal='.', index_col='Time')
data.head()

In [ ]:
data.tail()

In [ ]:
data.info()

In [ ]:
#plot_corrMatrix(data, 'pearson')

In [ ]:
#data[['Conc_P3-1','Conc_P3-2','Conc_P3-3','Conc_P3-4','MG_NIR_Salida_Ext']].plot()

Descarto los valores del 27 Jul 2021 16:00:00 en adelante porque se pierde valor de Conc_P3-4

In [ ]:
dataCrop = data.loc['2021-07-25 00:00:00':'2021-07-27 16:00:00']

In [ ]:
#dataCrop.plot()

In [ ]:
plot_corrMatrix(dataCrop, 'pearson')

Verifiqué que aún incluyendo retardo a todas las P3 la que mayor correlación obtiene es la Conc_P3-4 y la Conc_P3-5. Como entre ellas la correlación lineal también es alta, me quedo solo con la Conc_P3-4 para no incluir información redundante al modelo.

In [ ]:
dataCrop.drop(columns=['Conc_P3-1','Conc_P3-2','Conc_P3-3','Conc_P3-5','Conc_P3-6','Conc_P3-7','Conc_P3-8','Conc_P3-9','Conc_P3-10','Conc_P3-11'], inplace = True)

Descarto los TT-35-GO ya que está correlacionada linealmente con TT-35-GI.

In [ ]:
dataCrop.drop(columns = ['TT-35-GO'], inplace = True)

Elimino Valid_NIR_Salida_Ext ya que parecen estar representando situaciones reales. Se deben muestrear con mayor frecuencia. 

In [ ]:
dataCrop.drop(columns = ['Valid_NIR_Salida_Ext', 'Planta_Marcha'], inplace = True)

In [ ]:
plot_corrMatrix(dataCrop, 'pearson')

In [ ]:
make_mi_scores(dataCrop.drop(columns = 'MG_NIR_Salida_Ext'), dataCrop['MG_NIR_Salida_Ext'])

# CÁLCULO DE MEDIA MÓVIL

In [ ]:
dataCrop['MG_NIR_Salida_Ext_MVA'] = MVA(dataCrop.MG_NIR_Salida_Ext, 250) 

In [ ]:
dataCrop[['MG_NIR_Salida_Ext','MG_NIR_Salida_Ext_MVA']].plot()

In [ ]:
dataCrop['Vel_Extractor_MVA'] = MVA(dataCrop.Vel_Extractor, 100) 

In [ ]:
dataCrop[['Vel_Extractor','Vel_Extractor_MVA']].plot()

In [ ]:
dataCrop['Humedad_Semilla_MVA'] = MVA(dataCrop.Humedad_Semilla, 100) 

In [ ]:
dataCrop[['Humedad_Semilla','Humedad_Semilla_MVA']].plot()

In [ ]:
dataCrop['TT-35-GI_MVA'] = MVA(dataCrop['TT-35-GI'], 100) 

In [ ]:
dataCrop[['TT-35-GI','TT-35-GI_MVA']].plot()

In [ ]:
dataCrop['T_Lam_9_MVA'] = MVA(dataCrop['T_Lam_9'], 100) 

In [ ]:
dataCrop[['T_Lam_9_MVA','T_Lam_9']].plot()

In [ ]:
dataCrop['T_Mat_entrada_MVA'] = MVA(dataCrop['T_Mat_entrada'], 100) 

In [ ]:
dataCrop[['T_Mat_entrada_MVA','T_Mat_entrada']].plot()

In [ ]:
dataCrop['T_Solvente_MVA'] = MVA(dataCrop['T_Solvente'], 100) 

In [ ]:
dataCrop[['T_Solvente','T_Solvente_MVA']].plot()

In [ ]:
dataCrop['Conc_P3-4_MVA'] = MVA(dataCrop['Conc_P3-4'], 300) 

In [ ]:
dataCrop[['Conc_P3-4_MVA','Conc_P3-4']].plot()

In [ ]:
plot_corrMatrix(dataCrop, 'pearson')

In [ ]:
make_mi_scores(dataCrop.drop(columns = 'MG_NIR_Salida_Ext_MVA'), dataCrop['MG_NIR_Salida_Ext_MVA'])

# DEFINICIÓN DE DATASET

En base a los resultados obtenidos por correlación (Pearson, Kendall y Spearman) y el criterio de la información mutua, selecciono el siguiente conjunto de variables:


**LT - 17**

**Concentración P3-4 MVA**

**Concentración P-1**

**Temperatura laminado 9 MVA**

**Humedad de semilla MVA**

**Velocidad del extractor**

**Temperatura del material de entrada MVA**

**Temperatura del solvente**

In [ ]:
data_set = dataCrop[['LT-17','Conc_P3-4_MVA','T_Lam_9_MVA','Conc_P1','Humedad_Semilla_MVA','Vel_Extractor','T_Mat_entrada_MVA','TT-35-GI','MG_NIR_Salida_Ext_MVA','T_Solvente_MVA']][301:-301]

In [ ]:
plot_corrMatrix(data_set, 'pearson')

## Estandarización

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler()

In [ ]:
# Guardo el índice
index = data_set.index

In [ ]:
data_set = pd.DataFrame(scaler.fit_transform(data_set), columns = data_set.columns)

In [ ]:
data_set['Time'] = index

In [ ]:
make_mi_scores(data_set.drop(columns = 'MG_NIR_Salida_Ext_MVA'), data_set['MG_NIR_Salida_Ext_MVA'])

## Creación de nuevas features

In [ ]:
#data_set['TempMat_/_HumedadMat'] = data_set['T_Mat_entrada_MVA'] / data_set['Humedad_Semilla_MVA']
#data_set['TempSol_/_HumedadMat'] = data_set['T_Solvente_MVA'] / data_set['Humedad_Semilla_MVA']
data_set['CP1_/_HumedadMat'] = data_set['Conc_P1'] / data_set['Humedad_Semilla_MVA']
data_set['CP1_/_TLam'] = data_set['Conc_P1'] / data_set['T_Lam_9_MVA']
data_set['CP34_/_CP1'] = data_set['Conc_P3-4_MVA'] / data_set['Conc_P1'] 
#data_set['CP34_/_TLam'] = data_set['Conc_P3-4_MVA'] / data_set['T_Lam_9_MVA']
#data_set['TT-35-GI_2'] = data_set['TT-35-GI'] ** 2
#data_set['TempMat_/_TempSol'] = data_set['T_Mat_entrada_MVA'] / data_set['T_Solvente_MVA']

In [ ]:
plot_corrMatrix(data_set, 'pearson')

In [ ]:
make_mi_scores(data_set.drop(columns = 'MG_NIR_Salida_Ext_MVA'), data_set['MG_NIR_Salida_Ext_MVA'])

In [ ]:
pd.plotting.scatter_matrix(data_set, figsize=(24,16))

In [ ]:
data_set.plot()

In [ ]:
data_set.to_csv('To_FS_20210913_2137_3T.csv')

# CORRIMIENTOS TEMPORALES

In [ ]:
graficos = True

In [ ]:
maximize_correlation(dataCrop['MG_NIR_Salida_Ext_MVA'], dataCrop['Humedad_Semilla_MVA'], 120)

In [ ]:
maximize_correlation(dataCrop['MG_NIR_Salida_Ext'], dataCrop['Q_Miscela'], 90)